In [59]:
## This Notebook takes raw IMDb title data, filters by movie, joins it with ratings data, filters
## for only movies with 100 or more total ratings, and separates by genre.

import pandas as pd
pd.set_option('display.max_columns', None)

ratingsDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.ratings.tsv'
titleDataset = '/Users/davidkatilius/Documents/Raw_Datasets/IMDB/title.basics.tsv'

dfTitles = pd.read_csv(titleDataset, sep='\t')
dfRatings = pd.read_csv(ratingsDataset, sep='\t')

/var/folders/vb/t35ftt3d30b0gf_9v37z7st80000gn/T/ipykernel_39387/3847628580.py:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTitles = pd.read_csv(titleDataset, sep='\t')


In [60]:
dfTitles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [61]:
dfRatings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1901
1,tt0000002,5.9,254
2,tt0000003,6.5,1694
3,tt0000004,5.7,166
4,tt0000005,6.2,2511


In [62]:
## Filtering out non-movies, and movies that do not have a startYear
dfMovies = dfTitles[(dfTitles['titleType'] == 'movie') & (dfTitles['startYear'] != '\\N')]
dfMovies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama


In [63]:
## Checking size of the resulting dataframe
dfMovies.shape

(535173, 9)

In [64]:
## Joining movie data with ratings data
dfRatedMovies = pd.merge(dfMovies, dfRatings, on = "tconst", how = "inner")
dfRatedMovies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5.3,198
1,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.5,14
2,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,778
3,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama,4.7,19
4,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,\N,\N,Drama,4.5,23


In [65]:
## The data is around half its original size, suggesting many movies have no ratings whatsoever
dfRatedMovies.shape

(279919, 11)

In [66]:
## Now filtering for movies which have been rated at least 100 times, trimming down the size even further
dfRated100Times = dfRatedMovies[(dfRatedMovies['numVotes'] >= 100)]
dfRated100Times.shape

(113213, 11)

In [67]:
## titleType and endYear are all 'movie' and null respectively, so it's safe to remove them using .pop. 
print(dfRated100Times.titleType.unique())
print(dfRated100Times.endYear.unique())

['movie']
['\\N']


In [68]:
dfRated100Times.pop('titleType')
dfRated100Times.pop('endYear')

0         \N
2         \N
46        \N
55        \N
56        \N
          ..
279902    \N
279904    \N
279913    \N
279914    \N
279915    \N
Name: endYear, Length: 113213, dtype: object

In [69]:
dfRated100Times.head(100)

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198
2,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778
46,tt0001892,Den sorte drøm,Den sorte drøm,0,1911,53,Drama,5.9,236
55,tt0002101,Cleopatra,Cleopatra,0,1912,100,"Drama,History",5.1,533
56,tt0002130,Dante's Inferno,L'Inferno,0,1911,71,"Adventure,Drama,Fantasy",7.0,2884
...,...,...,...,...,...,...,...,...,...
500,tt0006864,Intolerance,Intolerance: Love's Struggle Throughout the Ages,0,1916,163,"Drama,History",7.7,15506
504,tt0006886,Judex,Judex,0,1916,300,"Adventure,Crime",7.3,1003
513,tt0006997,The Warrior,Maciste alpino,0,1916,95,"Action,Drama,War",6.0,102
522,tt0007047,The Matrimaniac,The Matrimaniac,0,1916,46,"Comedy,Romance",6.3,624


In [70]:
## Splitting genre data into separate columns.
dfRated100Times[['genre1', 'genre2', 'genre3']] = dfRated100Times['genres'].str.split(pat=',', expand=True)

/var/folders/vb/t35ftt3d30b0gf_9v37z7st80000gn/T/ipykernel_39387/1368092370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRated100Times[['genre1', 'genre2', 'genre3']] = dfRated100Times['genres'].str.split(pat=',', expand=True)
/var/folders/vb/t35ftt3d30b0gf_9v37z7st80000gn/T/ipykernel_39387/1368092370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRated100Times[['genre1', 'genre2', 'genre3']] = dfRated100Times['genres'].str.split(pat=',', expand=True)
/var/folders/vb/t35ftt3d30b0gf_9v37z7st8

In [22]:
dfRated100Times.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,genre1,genre2,genre3
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,Romance,None,None
2,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Action,Adventure,Biography
46,tt0001892,Den sorte drøm,Den sorte drøm,0,1911,53,Drama,5.9,236,Drama,None,None
55,tt0002101,Cleopatra,Cleopatra,0,1912,100,"Drama,History",5.1,533,Drama,History,None
56,tt0002130,Dante's Inferno,L'Inferno,0,1911,71,"Adventure,Drama,Fantasy",7.0,2884,Adventure,Drama,Fantasy


In [71]:
dfRated100Times_melted = pd.melt(dfRated100Times, id_vars=['tconst','primaryTitle','originalTitle','isAdult','startYear','runtimeMinutes','genres','averageRating','numVotes'])
dfRated100Times_melted = dfRated100Times_melted.sort_values(by=['tconst'])
dfRated100Times_melted.pop('variable')


0         genre1
226426    genre3
113213    genre2
1         genre1
226427    genre3
           ...  
113211    genre1
226424    genre2
226425    genre2
113212    genre1
339638    genre3
Name: variable, Length: 339639, dtype: object

In [77]:
dfRated100Times_melted.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,value
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,Romance
226426,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,None
113213,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,None
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Action
226427,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Biography


In [87]:
dfRated100Times_melted2 = dfRated100Times_melted2.dropna(subset='value')
dfRated100Times_melted2.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,value
0,tt0000009,Miss Jerry,Miss Jerry,0,1894,45,Romance,5.3,198,Romance
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Action
226427,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Biography
113214,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,778,Adventure
2,tt0001892,Den sorte drøm,Den sorte drøm,0,1911,53,Drama,5.9,236,Drama


In [88]:
## Popping primaryTitle and genres columns
dfRated100Times_melted2.pop('primaryTitle')
dfRated100Times_melted2.pop('genres')
dfRated100Times_melted2.head()

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,value
0,tt0000009,Miss Jerry,0,1894,45,5.3,198,Romance
1,tt0000574,The Story of the Kelly Gang,0,1906,70,6.0,778,Action
226427,tt0000574,The Story of the Kelly Gang,0,1906,70,6.0,778,Biography
113214,tt0000574,The Story of the Kelly Gang,0,1906,70,6.0,778,Adventure
2,tt0001892,Den sorte drøm,0,1911,53,5.9,236,Drama


In [89]:
dfRated100Times_melted2.rename(columns = {'value':'genre'}, inplace = True)


In [90]:
## Exporting final result to CSV so it can be visualized with Tableau
dfRated100Times_melted2.to_csv(r'movies_with_100_ratings.csv', index=False)